In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import gc
import time
from tqdm import tqdm
import sys
import requests
import json
import re
import warnings
warnings.filterwarnings('ignore')

In [26]:
# test_data = pd.read_csv('../data/test.csv')
train_data = pd.read_csv('../data/train.csv')

In [2]:
#　经纬度解析　
#encoding=utf8
import json
import urllib.request
import openpyxl
import xlwt
import time
import datetime
#基于百度地图API下的经纬度信息来解析地理位置信息
def getlocation(lng,lat):
    '''
    
    注意，需要将ｕｒｌ中（你的ａｋ密码）填上自己在百度开发者平台申请的ａｋ
    认证个人开发者，每天会有３０万次的访问次数限制
    百度开发者平台网址：http://lbsyun.baidu.com/index.php?title=%E9%A6%96%E9%A1%B5
    
    '''
    url = 'http://api.map.baidu.com/reverse_geocoding/v3/?ak=(你的ａｋ密码)&output=json&coordtype=wgs84ll&location=' + lat + ','+ lng
    # try except 防止一次请问请求不到，如果超时，则等待５秒访问第二次，还超时，继续等待５秒再进行访问，一般两次就会得到请求
    try:
        req = urllib.request.urlopen(url,timeout = 5)  # json格式的返回数据
    except:
        time.sleep(5)
        try:
            req = urllib.request.urlopen(url,timeout = 5)
        except:
            time.sleep(5)
            try:
                req = urllib.request.urlopen(url)
            except:
                pass
            
    res = req.read().decode("utf-8")  # 将其他编码的字符串解码成unicode
    return json.loads(res)

def jsonFormat(lng,lat):
    
    str_d = getlocation(lng,lat)
    dictjson={}#声明一个字典
    #get()获取json里面的数据
    jsonResult = str_d.get('result')
    while(jsonResult == None):
        str_d = getlocation(lng,lat)
        dictjson={}#声明一个字典
        #get()获取json里面的数据
        jsonResult = str_d.get('result')
        
   
    address = jsonResult.get('addressComponent')
    #国家
    country = address.get('country')
    #国家编号（0：中国）
    country_code = address.get('country_code')
    #省
    province = address.get('province')
    #城市
    city = address.get('city')
    #城市等级
    city_level = address.get('city_level')
    #县级
    district = address.get('district')
    #把获取到的值，添加到字典里（添加）
    # dictjson['country']=country
    # dictjson['country_code'] = country_code
    dictjson['province'] = province
    dictjson['city'] = city
    dictjson['district'] = district
    return dictjson


# train经纬度反解地址
## 百度ａｐｉ有限制一天３０万次的访问

In [27]:
df_train = train_data.copy()
df_train = df_train.drop_duplicates(['lng','lat']).reset_index(drop = True)
dis_result = pd.DataFrame()
print(train_data.shape[0])
print(df_train.shape[0])

11376681
369682


In [ ]:
# 百度ａｐｉ一天可一访问３０万次

begin=datetime.datetime.now()
for i in range(df_train.shape[0]):  # 打印1-2行，1-2列中的内容

    lng = df_train.loc[i,'lng']
    lat = df_train.loc[i,'lat']
    temp_dis_result = jsonFormat(str(lng),str(lat))
    print(i,temp_dis_result)
    dis_result.loc[i,'lng_lat'] = str(lng)+'_'+str(lat)
    dis_result.loc[i,'province'] = temp_dis_result['province']
    dis_result.loc[i,'city'] = temp_dis_result['city']
    dis_result.loc[i,'district'] = temp_dis_result['district']

end=datetime.datetime.now()
dur=end-begin
print(begin)
print(end)
print(dur)
dis_result.to_csv('train_lng_lat_reverse.csv',index = False)


# test 经纬度反解地址

In [ ]:
df_test = test_data.copy()
df_test = df_test.drop_duplicates(['lng','lat']).reset_index(drop = True)
dis_result = pd.DataFrame()
print(test_data.shape[0])
print(df_test.shape[0])

In [ ]:
begin=datetime.datetime.now()
for i in range(df_test.shape[0]):  # 打印1-2行，1-2列中的内容

    lng = df_train.loc[i,'lng']
    lat = df_train.loc[i,'lat']
    temp_dis_result = jsonFormat(str(lng),str(lat))
    print(i,temp_dis_result)
    dis_result.loc[i,'lng_lat'] = str(lng)+'_'+str(lat)
    dis_result.loc[i,'province'] = temp_dis_result['province']
    dis_result.loc[i,'city'] = temp_dis_result['city']
    dis_result.loc[i,'district'] = temp_dis_result['district']

end=datetime.datetime.now()
dur=end-begin
print(begin)
print(end)
print(dur)
dis_result.to_csv('test_lng_lat_reverse.csv',index = False)


In [2]:
a = pd.read_csv('../pickle/test_lng_lat_reverse.csv')

In [3]:
a.head()

,lng_lat,province,city,district
0,116.750876_36.56831,山东省,济南市,长清区
1,116.750867_36.56832,山东省,济南市,长清区
2,116.75086599999999_36.568292,山东省,济南市,长清区
3,116.75088000000001_36.5683,山东省,济南市,长清区
4,120.719481_30.753226,浙江省,嘉兴市,南湖区
